In [1]:
ls

0_To generate n-best hypothesis.ipynb    torgo.csv
1_To generate audio_features.ipynb       torgo_test_single.ipynb
2_finetune_whisper_on_torgo-Copy1.ipynb  Untitled.ipynb
3_prepare_for_torgo_baseline.ipynb       whisper-tiny/
output/


In [1]:
pwd

'/work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA/data_preparation'

In [6]:
!sbatch tauto.bash

Submitted batch job 42755979


In [4]:
!squeue -u zhang.jinda1

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          42744107       gpu sys/dash zhang.ji  R    6:56:15      1 d1009
          42754428       gpu sys/dash zhang.ji  R      59:26      1 d1024
          42755966       gpu tauto.ba zhang.ji  R       0:17      1 d1013


In [5]:
!scancel 42755966

In [2]:
import json
with open('training_args.json') as training_args_file:
    training_args_dict = json.load(training_args_file)

# Replace the default values with the values from the command line arguments
training_args_dict['learning_rate'] = learning_rate
training_args_dict['per_device_train_batch_size'] = train_batch_size
training_args_dict['per_device_eval_batch_size'] = eval_batch_size
training_args_dict['seed'] = seed
training_args_dict['gradient_accumulation_steps'] = gradient_accumulation_steps
training_args_dict['optim'] = optimizer
training_args_dict['lr_scheduler_type'] = lr_scheduler_type
training_args_dict['num_train_epochs'] = num_epochs

training_args = TrainingArguments(
    output_dir=model_local_path,
    hub_model_id=repo_name,
    **training_args_dict
)

NameError: name 'learning_rate' is not defined

In [6]:
import os
import glob

def delete_files_with_prefix(prefix):
    # Get the current working directory
    current_dir = os.getcwd()

    # Create a search pattern for files starting with the given prefix
    search_pattern = os.path.join(current_dir, f"{prefix}*")

    # Find all files matching the search pattern
    files_to_delete = glob.glob(search_pattern)

    # Iterate over the files and delete them
    for file_path in files_to_delete:
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

# Example usage
prefix = "slurm-4271395"  # Replace with your actual prefix
delete_files_with_prefix(prefix)

Deleted: /work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA/data_preparation/slurm-42713952.out


In [5]:
import json
with open('training_args.json') as training_args_file:
    training_args_dict = json.load(training_args_file)

In [6]:
training_args_dict

{'evaluation_strategy': 'steps',
 'per_device_train_batch_size': 4,
 'per_device_eval_batch_size': 4,
 'gradient_accumulation_steps': 2,
 'eval_delay': 0,
 'learning_rate': 0.0001,
 'weight_decay': 0.005,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'max_steps': -1,
 'lr_scheduler_type': 'linear',
 'warmup_ratio': 0.0,
 'warmup_steps': 1000,
 'save_strategy': 'steps',
 'save_steps': 500,
 'save_total_limit': 3,
 'report_to': 'all',
 'seed': 42,
 'eval_steps': 1000,
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'optim_args': None,
 'adafactor': False,
 'group_by_length': True,
 'length_column_name': 'length',
 'push_to_hub': True,
 'hub_strategy': 'every_save'}

In [1]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load the tokenizer, retriever, and model
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", use_dummy_dataset=True)
model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)

# Define the input query
input_query = "What is the capital of France?"

# Tokenize the input query
input_ids = tokenizer(input_query, return_tensors="pt").input_ids

# Generate the response
output_ids = model.generate(input_ids)

# Decode the generated response
response = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

print("Query:", input_query)
print("Response:", response)

/work/van-speech-nlp/jindaznb/mmenv/lib/python3.9/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


ImportError: 
RagRetriever requires the faiss library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/facebookresearch/faiss/blob/master/INSTALL.md and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
